<a href="https://colab.research.google.com/github/DayvsonNascimento/alloy/blob/master/Avalia%C3%A7%C3%A3o_de_Sistemas_de_RI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import nltk
import re
import collections
import heapq
import time
import bisect
import random
import sys
import numpy as np
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')

data = pd.read_csv('https://raw.githubusercontent.com/DayvsonNascimento/ri_lab_07/master/results.csv')
articles_text = data['text']
json = pd.read_json('https://raw.githubusercontent.com/DayvsonNascimento/ri_lab_07/master/results_final.json')
feedback = {json['query'][i]:json['docs'][i] for i in range(10)}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Reconstruir o índice considerando o conjunto de dados que foi indicado.

In [0]:
tknz = RegexpTokenizer(r'\b[^\d\W]+')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}

def get_indexes(articles_text, count):
    for text in articles_text:
        words = [word for word in tknz.tokenize(text.lower())
               if not bool(re.search(r'\d', word))
               and word not in stopwords and len(word) >= 3]  
        count += 1
        for t in words:
            if t not in indexes.keys():
                indexes[t] = []
            indexes[t].append(count)

    for elem in indexes.items():
        d = dict(collections.Counter(elem[1]))
        indexes[elem[0]] = list(d.items())
       
        
get_indexes(articles_text, count = 0)

Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário

In [0]:
for element in indexes:
    k = len(indexes[element])
    IDF = round(np.log((articles_text.count() + 1) / k), 2)
    
    indexes[element].append(IDF)

Implementação dos algoritmos:

In [0]:
"""
Implementação do algoritmo que segue a ideia de representação binária.
"""
def get_binary_order(query, text):
    query_terms = query.split()
    text_terms = text.split()
    counter = 0
  
    for term in query_terms:
        counter += (term in text_terms)
    
    return counter

In [0]:
"""
Implementação do algoritmo que segue a ideia de 'Term Frequency Weighting'.
"""
def get_TF_order(query, text):
    query_terms = query.split()
    text_terms = text.split()
    counter = 0
  
    for term in query_terms:
        counter += text_terms.count(term)
  
    return counter

In [0]:
"""
Implementação do algoritmo que segue a ideia 'Term Frequency Weighting' acrescentado do 
'inverse document frequency' de cada termo.
"""
def get_TF_IDF_order(query, text ):
    query_terms = query.split()
    text_terms = text.split()
    counter = 0
  
    for term in query_terms:
        cwd = text_terms.count(term)
        
        if term in indexes:
            counter += cwd * indexes[term][-1]
  
    return round(counter, 2)

In [0]:
"""
Implementação do algoritmo que segue a ideia do BM@5.
"""
def get_BM25_order(term, text, k):
    terms = term.split()
    text = text.split()
    counter = 0
  
    terms = [term for term in terms if term in text]
    
    for term in terms:
        cwd = text.count(term)
        dfw = 0
        
        if term in indexes:
            dfw = len(indexes[term][:-1])
        
        if(dfw != 0 ):
            counter += (((k+1) * cwd) / (cwd + k)) * np.log10(((articles_text.count() + 1) / dfw))
  
    return round(counter, 2)

In [0]:
"""
Função auxiliar para montar os arrays com os resultados para uma determinada query utilizando
os algoritmos do modelo vetorial i
"""
def get_models_results(query, n):
    binary_representation,  tf_representation, tfidf_representation, bm25_representation = [], [], [], []
    begin = 0
    
    for text in articles_text:
        text = text.lower()
        
        '''
        insere os resultados obtidos para as diferentes versões do modelo vetorial na posição 
        correta do array, começando da posição indicada pela variável 'begin'.
        '''
        bisect.insort(binary_representation, (get_binary_order(query, text), begin))
        bisect.insort(tf_representation, (get_TF_order(query, text), begin))
        bisect.insort(tfidf_representation, (get_TF_IDF_order(query, text), begin))
        bisect.insort(bm25_representation, (get_BM25_order(query, text, 10), begin))
        
        begin += 1
    
    #invertendo o array e pegando os n primeiros, pois são os que maiores scores.
    binary_representation = binary_representation[::-1][:n]
    tf_representation = tf_representation[::-1][:n]
    tfidf_representation = tfidf_representation[::-1][:n]
    bm25_representation = bm25_representation[::-1][:n]
        
    return binary_representation, tf_representation, tfidf_representation, bm25_representation

Q1: Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento. Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank 

Para esta questão, foi escolhida a pesquisa por "milícia maduro".

A url do documento escolhido é: https://brasil.elpais.com/brasil/2019/03/29/internacional/1553824596_128119.html, que corresponde ao documento de número '98', de título 'Milícia  o exército popular a serviço de Maduro'.

In [42]:
doc_number = 98
doc = data.loc[doc_number]

print(doc.url, "\n", doc.title)

https://brasil.elpais.com/brasil/2019/03/29/internacional/1553824596_128119.html 
 Milícia  o exército popular a serviço de Maduro


In [0]:
query = "milicia maduro"

binary_representation1, tf_representation1, tfidf_representation1, bm25_representation1 = get_models_results(query, 10)

Tabela dos resultados obtidos com os diferentes modelos

In [44]:
models_results = pd.DataFrame()

models_results['Binary'] = binary_representation1
models_results['TF'] = tf_representation1
models_results['TF-IDF'] = tfidf_representation1
models_results['BM25'] = bm25_representation1

models_results

,Binary,TF,TF-IDF,BM25
0,"(1, 246)","(10, 149)","(30.4, 149)","(7.25, 149)"
1,"(1, 182)","(5, 98)","(15.2, 98)","(4.84, 98)"
2,"(1, 152)","(5, 64)","(15.2, 64)","(4.84, 64)"
3,"(1, 149)","(4, 246)","(12.16, 246)","(4.14, 246)"
4,"(1, 98)","(4, 87)","(12.16, 87)","(4.14, 87)"
5,"(1, 96)","(2, 82)","(6.08, 82)","(2.42, 82)"
6,"(1, 87)","(2, 62)","(6.08, 62)","(2.42, 62)"
7,"(1, 82)","(1, 182)","(3.04, 182)","(1.32, 182)"
8,"(1, 64)","(1, 152)","(3.04, 152)","(1.32, 152)"
9,"(1, 62)","(1, 96)","(3.04, 96)","(1.32, 96)"


Reciprocal Rank

In [45]:
##Valores calculados com base na posição que o doc selecionado (98) aparece no ranking de cada algoritmo.
binary,tf, tf_idf, bm25 = 1/5, 1/2, 1/2, 1/2

rank= pd.DataFrame()
rank["Model"] = ['Binary', 'TF', 'TF-IDF', 'BM25'] 
rank['Result'] = [binary, tf, tf_idf, bm25]

rank

,Model,Result
0,Binary,0.2
1,TF,0.5
2,TF-IDF,0.5
3,BM25,0.5


Representação binária:

O binary retornou o documento esperado em 5ª posiçao com o Reciprocal Rank 1/5 = 0.2, vale ressaltar que o primeiro colocado, o documento de número 246 não fala em milícias, apenas de Maduro, sendo assim foi um resultado um pouco fora do esperado.

TF, TF-IDF, BM25:

Os modelos TF, TF-IDF e BM25 retornaram o documento esperado em segunda posição, com o Reciprocal Rank 1/2 = 0.5. O  documento de número 149 que foi retornado em primeiro posição nestes modelos , também fala bastante em Maduro e em milícias, o que justifica sua colocação. Portanto, o resultado obtido ao utilizar esses três algoritmos pode ser considerado bastante satisfatório.

Q2: A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante.

Funções auxiliares:

In [0]:
def getDocsIndexes(docs):
    array = []
    
    for i in range(len(docs)):
        array.append(docs[i][1])
        
    return array

def getIntersection(l1, l2):
    return len(list(set(l1) & set(l2))) 

def getUnion(l1, l2):
    return len(list(set(l1) | set(l2)))

def getRelevantDocs(query):
    docs = []
        
    for doc in feedback[query]:
        original_doc = data.loc[data.url == doc['URL']]
        docs.append(original_doc.index[0])
        
    return docs
    
def getAP(query):
    selected_docs = getRelevantDocs(query)

    binary, tf, tfidf, bm25 = get_models_results(query, 5)
    
    ##extracting only the docs indexes from the results.
    binary, tf, tf_idf, bm25 = getDocsIndexes(binary), getDocsIndexes(tf), getDocsIndexes(tfidf), getDocsIndexes(bm25)

    ##calcutaling the ap values for the algorithms.
    binary_value = getIntersection(binary, selected_docs) / len(binary)
    tf_value = getIntersection(tf, selected_docs) / len(tf)
    tfidf_value = getIntersection(tf_idf, selected_docs) / len(tf_idf)
    bm25_value = getIntersection(bm25, selected_docs) / len(bm25)

    return binary_value, tf_value, tfidf_value, bm25_value

Cálculo do MAP:

In [0]:
def getMAP(queries):
    binary_algorithm_sum, tf_algorithm_sum, tfidf_algorithm_sum, bm25_algorithm_sum = 0, 0, 0, 0
  
    for query in queries:
        ap_value_binary, ap_value_tf, ap_value_tfidf, ap_value_bm25 = getAP(query)
    
        binary_algorithm_sum += ap_value_binary
        tf_algorithm_sum += ap_value_tf
        tfidf_algorithm_sum += ap_value_tfidf
        bm25_algorithm_sum += ap_value_bm25

    query_length = len(queries)
    
    #calcutaling the MAP values for the algorithms.
    binary_value = binary_algorithm_sum /  query_length
    tf_value = tf_algorithm_sum / query_length
    tfidf_value = tfidf_algorithm_sum / query_length
    bm25_value = bm25_algorithm_sum / query_length
    
    return binary_value, tf_value, tfidf_value, bm25_value

In [0]:
binary_map_value, tf_map_value, tfidf_map_value, bm25_map_value = getMAP(feedback.keys())

Representação binária:

In [49]:
binary_map_value

0.1

O valor do MAP obtido utilizando os documentos selecionados e o algoritmo de representação binária foi de 0.1, um valor bastante baixo, tendo em vista que o valor do MAP vairia de 0 a 1.

TF:

In [50]:
tf_map_value

0.02

O valor do MAP obtido utilizando os documentos selecionados e o algoritmo TF foi de 0.02, um valor extremamente baixo, sendo o pior colocado dos algoritmos utilizados.

TF-IDF:

In [51]:
tfidf_map_value

0.18

O valor do MAP obtido utilizando os documentos selecionados e o algoritmo TF_IDF foi de 0.18, um valor também muito baixo, mas levemente maior do que os da Representação binária e o TF.

BM25:

In [52]:
bm25_map_value

0.2

O valor do MAP obtido utilizando os documentos selecionados e o algoritmo BM25 foi de 0.2, um valor baixo, porém o melhor colocado dentro os algoritmos utilizados. Vale lembrar que no lab anterior, o BM25 também se destacou por 'retornar melhores resultados' junto do TF-IDF, tanto que a diferença do MAP dos dois é de apenas 0.02.

Q3:Repita Q2 usando a avaliação multi-nível DCG. Utilize o campo "level" do gabarito para o cálculo do DCG e do idealDCG

Funções auxiliares a serem utilizadas no cálculo das métricas